In [ ]:
#https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

In [ ]:
pd.options.display.max_seq_items = 2000
pd.options.display.max_colwidth = 90
pd.options.display.max_rows = 999

In [1]:
issues = '../data/issue_all_jabref.csv'
dataIssues = pd.read_csv(issues,sep=',')

pull_request = '../data/pr_all_jabref.csv'
dataPR = pd.read_csv(pull_request,sep=',')
dataPRIssues = pd.read_csv(pull_request,sep=',') #copy to be added to issues

commits = '../data/commits_all_jabref.csv'
dataCommit = pd.read_csv(commits,sep=',', lineterminator='\n') #linetermination needed if file contains '\r' in places

In [2]:
#next cell will change this data
df = dataCommit
print(df.shape)
print(df.columns)
df.head(50)

(48872, 11)
Index(['issue_number', 'sha', 'message', 'author_name', 'date', 'additions',
       'deletions', 'changes', 'file_list', 'status', 'patch_text\r'],
      dtype='object')


,issue_number,sha,message,author_name,date,additions,deletions,changes,file_list,status,patch_text\r
0,1,a41477a3bf833b5e7eb4036cdbc7ff0d31f3715e,"Adding private fields to SQL scripts in order to export/import owner, timestamp, etc.",Igor Steinmacher,2013-10-23T19:13:12Z,15,1,16,src/java/net/sf/jabref/BibtexFields.java;src/java/net/sf/jabref/sql/SQLUtil.java,modified;modified,"@@ -434,6 +434,19 @@ public static String[] getAllFieldNames()\n {\n return runt..."
1,1,20d08479a8aacac1b672a45aed1db09f0e015f96,New entries translated to pt-BR,Igor Steinmacher,2013-10-23T23:37:02Z,93,93,186,src/resource/JabRef_pt_BR.properties;src/resource/Menu_pt_BR.properties,modified;modified,"@@ -1,5 +1,5 @@\n #!\r\n-#! created/edited by Popeye version 0.54 (popeye.sourceforge...."
2,1,0e365df01223ee8bac4bce8a7063065473550848,New entries translated to pt-BR,Igor Steinmacher,2013-10-24T19:22:22Z,147,147,294,src/resource/JabRef_pt_BR.properties,modified,"@@ -1322,7 +1322,7 @@ This_feature_generates_a_new_database_based_on_which_entries_are..."
3,1,929f3ff8c0949cfc74740e153c54b139b20a1812,Adds missing beta 2 separator to the CHANGELOG,Oliver Kopp,2013-10-27T13:14:44Z,1,0,1,src/txt/CHANGELOG,modified,"@@ -3,6 +3,7 @@\n - Fixed completeness indicator in main table for entries with cr..."
4,1,f87c07c35bed11351695142fe98720bbd57cbd88,Fixes string comparison at SQLUtil,Oliver Kopp,2013-10-27T13:17:00Z,1,2,3,src/java/net/sf/jabref/sql/SQLUtil.java,modified,"@@ -27,7 +27,6 @@\n import java.util.ListIterator;\n \n import net.sf.jabref.BibtexFie..."
5,1,4f229e2fbd390fdaa72953aaffa5d5afccc533eb,Update Japanese property files,Koji Yokota,2013-10-31T23:33:09Z,108,108,216,src/resource/JabRef_ja.properties;src/resource/Menu_ja.properties,modified;modified,"@@ -1831,6 +1831,8 @@ Legacy_file_fields=\u5ec3\u6b62\u3055\u308c\u305f\u30d5\u30a1\u3..."
6,1,dfc942ace050c8ffa58c04c2b70bc92709189e14,Merge branch 'beta_2.10',Oliver Kopp,2013-11-04T08:35:17Z,359,345,704,src/java/net/sf/jabref/BibtexFields.java;src/java/net/sf/jabref/sql/SQLUtil.java;src/r...,modified;modified;modified;modified;modified;modified;modified,"@@ -434,6 +434,19 @@ public static String[] getAllFieldNames()\n {\n return runt..."
7,1,c220c4b65aace94183d91e51316da596e03a351b,Merge branch 'master' of ssh://git.code.sf.net/p/jabref/code,Oliver Kopp,2014-02-12T12:09:43Z,0,0,0,NaN,NaN,\r
8,1,5bbb074058bf2bb7da889df3941bd20ed7b03e21,Fix GoogleScholarFetcher\n\nthe format of Google has changed and required an updated r...,Andreas Amann,2014-02-11T19:33:38Z,1,1,2,src/java/net/sf/jabref/imports/GoogleScholarFetcher.java,modified,"@@ -42,7 +42,7 @@ public class GoogleScholarFetcher implements PreviewEntryFetcher {\n..."
9,1,25887febb273fe9ce169e773de08df5c0d2c5e3a,Release of 2.10 beta 3,Oliver Kopp,2014-02-12T12:38:42Z,6,5,11,build.xml;src/txt/CHANGELOG;src/txt/README,modified;modified;modified,"@@ -35,9 +35,9 @@\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++..."


In [3]:
#drop any multiple commits for a pr, keeping only the last one (last date).
#assumes the commits are listed in order by date.
df.drop_duplicates(subset='issue_number',keep='last',inplace=True)

In [4]:
#converting Float to Int 
dataIssues['issue_number'] = dataIssues['issue_number'].fillna(0).astype(np.int64)

#renaming columns and excluding unecessary data

#commits needs only pr/issue number, commit author, commit date, and commit message.
dataCommit.columns = ['PR_Number','sha(not needed)', 'Commit_Message', 'Commit_Author', 'Commit_Date', 'additions(not needed)', 'deletions(not needed)', 'changes(not needed)', 'file_list(not needed)', 'status(not needed)', 'patch_text\r(not needed)']
dataCommit.drop(['sha(not needed)', 'additions(not needed)', 'deletions(not needed)', 'changes(not needed)', 'file_list(not needed)', 'status(not needed)', 'patch_text\r(not needed)'], axis=1, inplace=True)

#prs needs only pr/issue number, pr closed date, pr author, pr title, pr body, and pr comments.
##This data is missing pr author so author login will represent it instead
dataPR.columns = ['PR_Number', 'PR_Title', 'pr_author(empty)', 'PR_Author', 'PR_Closed_Date', 'PR_Body', 'PR_Comments']
del dataPR['pr_author(empty)']

#issues needs only pr/issue number, issue closed date, issue author, issue title, issue body, and issue comments
##issue author is also empty here
dataIssues.columns = ['PR_Number', 'Issue_Title', 'issue_author(empty)', 'Issue_Author', 'Issue_Closed_Date', 'Issue_Body', 'Issue_Comments']
del dataIssues['issue_author(empty)']

##only necesary if issues data does not already contain PRs
dataPRIssues.columns = ['PR_Number', 'Issue_Title', 'issue_author(empty)', 'Issue_Author', 'Issue_Closed_Date', 'Issue_Body', 'Issue_Comments']
del dataPRIssues['issue_author(empty)']

In [5]:
df = dataCommit
print(df.shape)
print(df.columns)
df.head(500)

(8431, 4)
Index(['PR_Number', 'Commit_Message', 'Commit_Author', 'Commit_Date'], dtype='object')


,PR_Number,Commit_Message,Commit_Author,Commit_Date
19,1,Merge branch 'sorting',Olaf Lenz,2014-03-12T09:16:45Z
21,2,Gradle build works basically. Reuses existing folder structure.,Simon Harrer,2014-03-12T17:20:08Z
22,3,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,2014-03-15T09:33:10Z
23,4,Spanish translation update,jtornero,2014-03-17T21:28:26Z
24,5,Update JabRef_in.properties\n\nIndonesian translation added,was123,2014-03-18T00:01:58Z
25,6,Fixed splash-dev.svg,Olaf Lenz,2014-03-19T19:41:29Z
26,7,I have implemented 2 features requested in Ticket #841.\n\n The user can now speci...,noravanq,2014-03-23T04:48:49Z
28,8,Erase fields listed in XMP Privacy Settings from PDF DocumentInformation when XMP-tagging,Adrian Daerr,2014-04-22T18:19:15Z
30,9,add CSIRO copyright 2014,Peter Ansell,2014-05-07T01:02:09Z
34,10,fix bug causing double entries.,Ling Wang,2014-05-20T02:56:41Z


In [6]:
#merging dataframes
pull_commit = pd.merge(dataPR, dataCommit, on=['PR_Number','PR_Number'])

pull_issue = pd.concat([dataPRIssues, dataIssues], axis=0)
pull_issue.sort_values(by='PR_Number', inplace=True)

In [7]:
pull_commit.to_csv('pull_commit.csv', sep=',', encoding='utf-8', index=False)

pull_issue.to_csv('issues.csv', sep=',', encoding='utf-8', index=False)


In [8]:
data_Issues = './issues.csv'
dataIssues = pd.read_csv(data_Issues, sep=',' )

data_PullCommit = './pull_commit.csv'
dataPullCommit = pd.read_csv(data_PullCommit, sep=',' )

In [9]:
dataIssues.head(100)

,PR_Number,Issue_Title,Issue_Author,Issue_Closed_Date,Issue_Body,Issue_Comments
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN
1,2,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN
2,3,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN
3,4,Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request...."
4,5,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN
5,6,Fixed splash-dev.svg,olenz,2014-03-19T19:44:35Z,NaN,NaN
6,7,I have implemented 2 features requested in Ticket #841.,noravanq,2014-03-23T04:53:18Z,"```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN
7,8,PDF-file metadata: Privacy Filtering all metadata,adaerr,2014-04-23T12:58:24Z,This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n
8,9,Support FindFullText with ACS DOIs,ansell,2014-05-06T06:36:03Z,Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN
9,10,try to fix some obvious bugs about `groups`,braindevices,2014-05-08T03:14:52Z,"Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...


In [10]:
dataPullCommit.head(100)

,PR_Number,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments,Commit_Message,Commit_Author,Commit_Date
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'sorting',Olaf Lenz,2014-03-12T09:16:45Z
1,2,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Gradle build works basically. Reuses existing folder structure.,Simon Harrer,2014-03-12T17:20:08Z
2,3,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,2014-03-15T09:33:10Z
3,4,Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,jtornero,2014-03-17T21:28:26Z
4,5,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN,Update JabRef_in.properties\n\nIndonesian translation added,was123,2014-03-18T00:01:58Z
5,6,Fixed splash-dev.svg,olenz,2014-03-19T19:44:35Z,NaN,NaN,Fixed splash-dev.svg,Olaf Lenz,2014-03-19T19:41:29Z
6,7,I have implemented 2 features requested in Ticket #841.,noravanq,2014-03-23T04:53:18Z,"```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN,I have implemented 2 features requested in Ticket #841.\n\n The user can now speci...,noravanq,2014-03-23T04:48:49Z
7,8,PDF-file metadata: Privacy Filtering all metadata,adaerr,2014-04-23T12:58:24Z,This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n,Erase fields listed in XMP Privacy Settings from PDF DocumentInformation when XMP-tagging,Adrian Daerr,2014-04-22T18:19:15Z
8,9,Support FindFullText with ACS DOIs,ansell,2014-05-06T06:36:03Z,Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN,add CSIRO copyright 2014,Peter Ansell,2014-05-07T01:02:09Z
9,10,try to fix some obvious bugs about `groups`,braindevices,2014-05-08T03:14:52Z,"Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...,fix bug causing double entries.,Ling Wang,2014-05-20T02:56:41Z


In [11]:
#result = pd.merge(user_usage, user_device[['use_id', 'platform', 'device']], on='use_id', how='outer',indicator=True)
result = pd.merge(dataIssues,
                 dataPullCommit,
                 on='PR_Number',
                 how='outer', 
                 indicator=True)

In [12]:
print(result.shape)
print(result.columns)
result.head(100)

(12527, 15)
Index(['PR_Number', 'Issue_Title', 'Issue_Author', 'Issue_Closed_Date',
       'Issue_Body', 'Issue_Comments', 'PR_Title', 'PR_Author',
       'PR_Closed_Date', 'PR_Body', 'PR_Comments', 'Commit_Message',
       'Commit_Author', 'Commit_Date', '_merge'],
      dtype='object')


,PR_Number,Issue_Title,Issue_Author,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments,Commit_Message,Commit_Author,Commit_Date,_merge
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'sorting',Olaf Lenz,2014-03-12T09:16:45Z,both
1,2,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Gradle build works basically. Reuses existing folder structure.,Simon Harrer,2014-03-12T17:20:08Z,both
2,3,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,2014-03-15T09:33:10Z,both
3,4,Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,jtornero,2014-03-17T21:28:26Z,both
4,5,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN,Update JabRef_in.properties\n\nIndonesian translation added,was123,2014-03-18T00:01:58Z,both
5,6,Fixed splash-dev.svg,olenz,2014-03-19T19:44:35Z,NaN,NaN,Fixed splash-dev.svg,olenz,2014-03-19T19:44:35Z,NaN,NaN,Fixed splash-dev.svg,Olaf Lenz,2014-03-19T19:41:29Z,both
6,7,I have implemented 2 features requested in Ticket #841.,noravanq,2014-03-23T04:53:18Z,"```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN,I have implemented 2 features requested in Ticket #841.,noravanq,2014-03-23T04:53:18Z,"```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN,I have implemented 2 features requested in Ticket #841.\n\n The user can now speci...,noravanq,2014-03-23T04:48:49Z,both
7,8,PDF-file metadata: Privacy Filtering all metadata,adaerr,2014-04-23T12:58:24Z,This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n,PDF-file metadata: Privacy Filtering all metadata,adaerr,2014-04-23T12:58:24Z,This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n,Erase fields listed in XMP Privacy Settings from PDF DocumentInformation when XMP-tagging,Adrian Daerr,2014-04-22T18:19:15Z,both
8,9,Support FindFullText with ACS DOIs,ansell,2014-05-06T06:36:03Z,Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN,Support FindFullText with ACS DOIs,ansell,2014-05-06T06:36:03Z,Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN,add CSIRO copyright 2014,Peter Ansell,2014-05-07T01:02:09Z,both
9,10,try to fix some obvious bugs about `groups`,braindevices,2014-05-08T03:14:52Z,"Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...,try to fix some obvious bugs about `groups`,braindevices,2014-05-08T03:14:52Z,"Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The opt

In [13]:
result.rename(columns={'_merge':'isPR'}, inplace=True) #_merge is a column created by pd.merge()
result['isPR'].value_counts()


isPR
both          8431
left_only     4096
right_only       0
Name: count, dtype: int64

In [14]:
result['isPR'] = result['isPR'].apply({'both': 1, 'left_only': 0, 'right_only':0}.get)



In [15]:
result.to_csv('merged_Data.csv', sep=',', encoding='utf-8', index=False)


In [16]:
data = './merged_Data.csv'
data = pd.read_csv(data, sep=',' )


In [17]:
print(data.shape)
data.head()

(12527, 15)


,PR_Number,Issue_Title,Issue_Author,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments,Commit_Message,Commit_Author,Commit_Date,isPR
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'sorting',Olaf Lenz,2014-03-12T09:16:45Z,1
1,2,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Gradle build works basically. Reuses existing folder structure.,Simon Harrer,2014-03-12T17:20:08Z,1
2,3,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,2014-03-15T09:33:10Z,1
3,4,Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,jtornero,2014-03-17T21:28:26Z,1
4,5,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN,Update JabRef_in.properties\n\nIndonesian translation added,was123,2014-03-18T00:01:58Z,1


In [18]:
sum(pd.isnull(data['Issue_Closed_Date']))
data_final = data[data['Issue_Closed_Date'].notna()]

In [19]:
data_final.to_csv('dataMerged_Final_only_closed.csv', sep=',', encoding='utf-8', index=False)

In [20]:
data = './dataMerged_Final_only_closed.csv'
data = pd.read_csv(data, sep=',' )

In [21]:
data.head(100)

,PR_Number,Issue_Title,Issue_Author,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments,Commit_Message,Commit_Author,Commit_Date,isPR
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'sorting',Olaf Lenz,2014-03-12T09:16:45Z,1
1,2,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Gradle build works basically. Reuses existing folder structure.,Simon Harrer,2014-03-12T17:20:08Z,1
2,3,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,2014-03-15T09:33:10Z,1
3,4,Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,jtornero,2014-03-17T21:28:26Z,1
4,5,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN,Update JabRef_in.properties\n\nIndonesian translation added,was123,2014-03-18T00:01:58Z,1
5,6,Fixed splash-dev.svg,olenz,2014-03-19T19:44:35Z,NaN,NaN,Fixed splash-dev.svg,olenz,2014-03-19T19:44:35Z,NaN,NaN,Fixed splash-dev.svg,Olaf Lenz,2014-03-19T19:41:29Z,1
6,7,I have implemented 2 features requested in Ticket #841.,noravanq,2014-03-23T04:53:18Z,"```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN,I have implemented 2 features requested in Ticket #841.,noravanq,2014-03-23T04:53:18Z,"```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN,I have implemented 2 features requested in Ticket #841.\n\n The user can now speci...,noravanq,2014-03-23T04:48:49Z,1
7,8,PDF-file metadata: Privacy Filtering all metadata,adaerr,2014-04-23T12:58:24Z,This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n,PDF-file metadata: Privacy Filtering all metadata,adaerr,2014-04-23T12:58:24Z,This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n,Erase fields listed in XMP Privacy Settings from PDF DocumentInformation when XMP-tagging,Adrian Daerr,2014-04-22T18:19:15Z,1
8,9,Support FindFullText with ACS DOIs,ansell,2014-05-06T06:36:03Z,Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN,Support FindFullText with ACS DOIs,ansell,2014-05-06T06:36:03Z,Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN,add CSIRO copyright 2014,Peter Ansell,2014-05-07T01:02:09Z,1
9,10,try to fix some obvious bugs about `groups`,braindevices,2014-05-08T03:14:52Z,"Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...,try to fix some obvious bugs about `groups`,braindevices,2014-05-08T03:14:52Z,"Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...,